In [1]:
import PIL
import numpy as np
import torch
import matplotlib.pyplot as plt
import vtk
from vtk.util.numpy_support import vtk_to_numpy
from tqdm import tqdm
from scipy.optimize import minimize
from vedo import *
import meshio
import seaborn as sns
import pyvista as pv
from data_processing.mesh_to_array import *
import pandas as pd
from matplotlib.colors import LinearSegmentedColormap
from hydra import compose, initialize
from omegaconf import OmegaConf
from utils.helper import make_cmap, get_all_cases
import fnmatch
import cv2
from data_processing.mesh_to_array import mesh_to_image
from data_processing.dataset_3d import make_npz_3d, load_dataset_3d, make_dataset_3d, make_npz_3d_multi, write_npz_case_3d
from data_processing.dataset_3d import change_rho
from data_processing.preprocessing import load_signals
# from data_processing.obj2py import read_get
from utils.helper import make_cmap

In [13]:
load_signals(dir = '/home/nibdombe/deep_eit/data/raw/case_TCIA_404_0/')

(array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         ...,
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.        ,
           0.        ,  0.        ]],
 
        [[ 0.        ,  0.        , 11.51216507, ...,  1.9046129 ,
           5.33557367,  0.        ],
         [ 0.        ,  0.        ,  0.        , ...,  0.37056428,
           0.69410956,  7.25867462],
         [11.51205826,  0.        ,  0.        , ...,  0.19454066,
           0.27288866,  1.93046832],
         ...,
         [ 1.90453196,  0.3704627

In [ ]:
def load_cfg(path):
    with initialize(version_base=None, config_path=path, job_name="test"):
        cfg = compose(config_name="config")
    return cfg

cmap = make_cmap()
cfg = load_cfg('cfg')

# Are the signals linear in conductivity?

In [ ]:
train_data = torch.load('data/datasets/train_dataset_3d.pt')

In [ ]:
signals_5 = []
signals_10 = []
signals_15 = []
signals_20 = []

for case in train_data.case_files:
    tmp = np.load('data/processed/3d_mask/'+case)
    rho = np.unique(tmp['targets'])[3]
    if rho == 1/5:
        signals_5.append(tmp['signals'])
    if rho == 1/10:
        signals_10.append(tmp['signals'])
    if rho == 1/15:
        signals_15.append(tmp['signals'])
    if rho == 1/20:
        signals_20.append(tmp['signals'])

In [ ]:
signals_5 = np.stack(signals_5, axis=0)
signals_10 = np.stack(signals_10, axis=0)
signals_15 = np.stack(signals_15, axis=0)
signals_20 = np.stack(signals_20, axis=0)

In [ ]:
data = np.load('/home/nibdombe/deep_eit/data/processed/3d_mask/case_TCIA_2_0/case_TCIA_2_0_15.npz')

In [ ]:
np.unique(data['targets'])

In [ ]:
signals_5.shape

In [ ]:
np.corrcoef(np.vstack((signals_5[:,0].reshape(-1), signals_10[:,0].reshape(-1), signals_15[:,0].reshape(-1), signals_20[:,0].reshape(-1))))

In [ ]:
np.quantile(signals_5.reshape(-1), q=[0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99])

In [ ]:
signals_5.shape

In [ ]:
fig, ax = plt.subplots()
rhos = np.array([5.,10.,15.,20.])
ax.scatter(rhos, np.array([signals_5[0,0,0],signals_10[0,0,0],signals_15[0,0,0],signals_20[0,0,0]]))
ax.scatter(rhos, np.array([signals_5[0,0,1],signals_10[0,0,1],signals_15[0,0,1],signals_20[0,0,1]]), c='green')
ax.scatter(rhos, np.array([signals_5[0,0,2],signals_10[0,0,2],signals_15[0,0,2],signals_20[0,0,2]]), c='red')
ax.scatter(rhos, np.array([signals_5[0,0,3],signals_10[0,0,3],signals_15[0,0,3],signals_20[0,0,3]]), c='orange')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Perform linear regression using numpy
def linear_regression(x, y):
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    return m, c

def make_reg(x,y):
    # Perform linear regression on filtered data
    m_final, c_final = linear_regression(x, y)
    y_pred_final = m_final * x + c_final

    # Plot data points
    plt.scatter(x, y, color='blue', label='Data points')

    # Plot regression line
    plt.plot(x, y_pred_final, color='red', label=f'Regression line: y={m_final:.2f}x+{c_final:.2f}')

    # Add labels, legend, and estimated parameters
    plt.xlabel('Lung Resistivity')
    plt.ylabel('Voltages')
    plt.legend()

    # Show estimated parameters on the plot
    plt.text(0.05, 0.95, f'y = {m_final:.2f}x + {c_final:.2f}', transform=plt.gca().transAxes,
            fontsize=12, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.5))

    # Show plot
    plt.show()


In [ ]:
for i in range(100):
    # Create example data
    x = rhos
    y = np.array([signals_5[0,0,i],signals_10[0,0,i],signals_15[0,0,i],signals_20[0,0,i]])

    make_reg(x,y)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Create example data
x=signals_5[:,0,0].reshape(-1)
y=signals_20[:,0,0].reshape(-1)

# Determine the 90th percentile for both arrays
x_threshold = np.percentile(x, 90)
y_threshold = np.percentile(y, 90)

# Filter out values higher than the 90th percentile in either array
mask = (x <= x_threshold) & (y <= y_threshold)
x_filtered = x[mask]
y_filtered = y[mask]

# Perform linear regression using numpy
def linear_regression(x, y):
    A = np.vstack([x, np.ones(len(x))]).T
    m, c = np.linalg.lstsq(A, y, rcond=None)[0]
    return m, c

# Perform linear regression on filtered data
m_final, c_final = linear_regression(x_filtered, y_filtered)
y_pred_final = m_final * x_filtered + c_final

# Plot data points
plt.scatter(x_filtered, y_filtered, color='blue', label='Data points')

# Plot regression line
plt.plot(x_filtered, y_pred_final, color='red', label=f'Regression line: y={m_final:.2f}x+{c_final:.2f}')

# Add labels, legend, and estimated parameters
plt.xlabel('X values')
plt.ylabel('Y values')
plt.title('Linear Regression after Filtering')
plt.legend()

# Show estimated parameters on the plot
plt.text(0.05, 0.95, f'y = {m_final:.2f}x + {c_final:.2f}', transform=plt.gca().transAxes,
         fontsize=12, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.5))

# Show plot
plt.show()


In [ ]:
plt.scatter(signals_5[0,1], signals_10[0,1])

# Make 3d points

In [ ]:
cfg.data.processed_data_folder = 'data/processed/3d'
cases = get_all_cases(cfg, base_dir='', use_raw=True)

In [ ]:
cases = ['case_TCIA_110_0']

In [ ]:
write_npz_case_3d(cases[0], resolution=512, electrode_resolution=512, all_signals=True, points_3d=True)

In [ ]:
train_dataset, val_dataset, test_dataset = load_dataset_3d(cases,
                write_dataset=False, write_npz=True, points_3d=True, processed_data_folder='data/processed/3d', overwrite_npz=False, 
                base_dir='', resolution=512, electrode_resolution=512, all_signals=True)

In [ ]:
train_dataset.training = False
points_max = []
points_min = []
for p, s, e, m, t in tqdm(train_dataset):
    points_max.append(p.max(dim=0).values)
    points_min.append(p.min(dim=0).values)

In [ ]:
print(torch.stack(points_max).max(dim=0),
      torch.stack(points_min).min(dim=0))

In [ ]:
tmp = torch.stack(points_max)

In [ ]:
tmp = tmp + 0.2 

In [ ]:
torch.any(torch.logical_or(tmp > 1, tmp < -1), dim=1)

In [ ]:
torch.any((tmp > 1) + (tmp < -1), dim=1)

In [ ]:
torch.where(tmp > 1 or tmp < -1)

In [ ]:
fig, axes = plt.subplots(1,8)
tmp_points = train_dataset.points.reshape(8, 4, -1, 8, 3)
for i in range(8):
    # axes[i].imshow(train_dataset.targets[4,0,:,:,i], cmap=cmap)
    axes[i].scatter(tmp_points[0,0,:,i,0], tmp_points[0,0,:,i,1], marker='.', c=(train_dataset.targets[0,0,:,:,i].reshape(-1)), cmap=cmap, s=1)


In [ ]:
test = np.load('/home/nibdombe/deep_eit/data/processed/case_TCIA_2_0.npz')

In [ ]:
targets = test['targets']
targets = np.moveaxis(targets.reshape(4,4,512,512,1),0,1)

In [ ]:
test['points'].shape

In [ ]:
plt.imshow(targets[0,2], cmap=cmap)

In [ ]:
targets.shape

# Show slice from .nas

In [ ]:
from data_processing.preprocessing import load_mask_target_electrodes
import matplotlib.pyplot as plt

In [ ]:
mesh_data = load_mask_target_electrodes('/home/nibdombe/deep_eit/data/raw/case_TCIA_250_0', resolution=512)

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(30, 10))
for i in range(4):
    axes[0,i].imshow(mesh_data[0][i])
    axes[0,i].axis('off')
    axes[0,i].scatter(mesh_data[-1][i][:,0], mesh_data[-1][i][:,1], marker='x', c='r')
    axes[1,i].imshow(mesh_data[1][i])
    axes[1,i].axis('off')
    axes[1,i].scatter(mesh_data[-1][i][:,0], mesh_data[-1][i][:,1], marker='x', c='r')



# Check data from .npz files

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Individual case

In [ ]:
data = np.load('/home/nibdombe/deep_eit/data/processed/3d/case_TCIA_2_0/case_TCIA_2_0_20.npz')
print(
    data['targets'].shape,
    data['points'].shape,
    data['electrodes'].shape,
    data['masks'].shape,
    data['signals'].shape
)

In [ ]:
targets = data['targets']
points = data['points']
electrodes = data['electrodes']

In [ ]:
fig, axes = plt.subplots(3, 2, figsize=(10, 3))
for i, ax in enumerate(axes.flatten()):
    # ax.imshow(targets[i], cmap=cmap)
    ax.axis('off')
    ax.scatter(points[i][:,0], points[i][:,1], marker='.', c=(targets[i].reshape(-1)), cmap=cmap, s=1)
    # ax.scatter(electrodes[i][0,0],electrodes[i][0,1], marker='x', c='r', s=50)

## Check all cases

In [ ]:
path = '/home/nibdombe/deep_eit/data/processed/'
cases = os.listdir(path)
cases = [case for case in cases if fnmatch.fnmatch(case, 'case_TCIA*')]


### Electrode position

In [ ]:
for i, case in enumerate(tqdm(cases)):
    case_path = os.path.join(path, case) 
    data = np.load(case_path)
    targets = data['targets']
    points = data['points']
    electrodes = data['electrodes']
    idx = np.where(targets[0].reshape(-1) != 0.)[0]
    rel_points = points[0][idx]
    min_x = np.min(rel_points[:,0])
    max_x = np.max(rel_points[:,0])
    min_y = np.min(rel_points[:,1])
    max_y = np.max(rel_points[:,1])
    
    min_x_electrodes = np.min(electrodes[0][:,0])
    max_x_electrodes = np.max(electrodes[0][:,0])
    min_y_electrodes = np.min(electrodes[0][:,1])
    max_y_electrodes = np.max(electrodes[0][:,1])

    distances = np.array([min_x - min_x_electrodes,
        max_x - max_x_electrodes,
        min_y - min_y_electrodes,
        max_y - max_y_electrodes])
    if np.any(np.abs(distances) > 0.01):
        fig, ax = plt.subplots(1, 1, figsize=(3, 3))
        ax.axis('off')
        ax.scatter(points[0][:,0], points[0][:,1], marker='.', c=(targets[0].reshape(-1)), cmap=cmap, s=1)
        ax.scatter(electrodes[0][:,0],electrodes[0][:,1], marker='x', c='r', s=50)
        ax.set_title(case)
        plt.show()
        plt.close()

# Check signals scale

In [ ]:
for i, case in enumerate(tqdm(cases)):
    case_path = os.path.join(path, case) 
    data = np.load(case_path)
    signals = data['targets']
    plt.hist(signals[0].flatten())
    plt.show()

## Check all cases visually

In [ ]:
for i, case in enumerate(cases):
    case_path = os.path.join(path, case) 
    data = np.load(case_path)
    targets = data['targets']
    points = data['points']
    electrodes = data['electrodes']
    fig, ax = plt.subplots(1, 1, figsize=(3, 3))
    ax.axis('off')
    ax.scatter(points[0][:,0], points[0][:,1], marker='.', c=(targets[0].reshape(-1)), cmap=cmap, s=1)
    ax.scatter(electrodes[0][:3,0],electrodes[0][:3,1], marker='x', c='r', s=50)
    ax.set_title(case)
    ax.set_xlim(-1,1)
    ax.set_ylim(-1,1)
    plt.show()
    plt.close()


# Check dataset files

In [ ]:
train_dataset = torch.load('data/datasets/train_dataset.pt')

In [ ]:
train_dataset.cases

In [ ]:
len(train_dataset.cases)

In [ ]:
for i in range(len(train_dataset)):
    targets = train_dataset.targets[i]
    points = train_dataset.points[i]
    electrodes = train_dataset.electrodes[i,:,0,0]
    case = train_dataset.cases[i]
    fig, ax = plt.subplots(1, 1, figsize=(3, 3))
    ax.axis('off')
    ax.scatter(points[:,0], points[:,1], marker='.', c=(targets.reshape(-1)), cmap=cmap, s=1)
    ax.scatter(electrodes[:3,0],electrodes[:3,1], marker='x', c='r', s=50)
    ax.set_title(case)
    ax.set_xlim(-1,1)
    ax.set_ylim(-1,1)
    plt.show()
    plt.close()
    if i==10:
        break

In [ ]:
electrodes.shape

In [ ]:
case = 'case_TCIA_25_0.npz'
case_path = os.path.join(path, case) 
data = np.load(case_path)
targets = data['targets']
points = data['points']
electrodes = data['electrodes']
fig, ax = plt.subplots(1, 1, figsize=(3, 3))
ax.scatter(points[0][:,0], points[0][:,1], marker='.', c=(targets[0].reshape(-1)), cmap=cmap, s=1)
ax.scatter(electrodes[0][:16,0], electrodes[0][:16,1], marker='x', c='r', s=50)
ax.set_title(case)
plt.show()
plt.close()


In [ ]:
electrodes[0]

In [ ]:
for i, case in enumerate(cases):
    case_path = os.path.join(path, case) 
    data = np.load(case_path)
    targets = data['targets']
    points = data['points']
    electrodes = data['electrodes']
    fig, ax = plt.subplots(1, 1, figsize=(3, 3))
    ax.axis('off')
    ax.scatter(points[0][:,0], points[0][:,1], marker='.', c=(targets[0].reshape(-1)), cmap=cmap, s=1)
    ax.scatter(electrodes[0][:3,0],electrodes[0][:3,1], marker='x', c='r', s=50)
    ax.set_title(case)
    plt.show()
    plt.close()


In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(30, 10))
for i, ax in enumerate(axes.flatten()):
    ax.scatter(points[i][:,0], points[i][:,1], marker='.', c=(targets[i].reshape(-1)))
    ax.axis('off')
    ax.scatter(electrodes[i][:,0], electrodes[i][:,1], marker='x', c='r')

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

# Define the colors and their positions in the colormap
colors = [(1., 1., 1),   # blue
          (0.1, 0.1, 1), # light blue
          (1, 0.8, 0.4), # yellow
          (0.8, 0, 0)]     # red
positions = [0.0, 0.05, 0.2, 1.0]

# Create the colormap
cmap = LinearSegmentedColormap.from_list('custom_colormap', list(zip(positions, colors)))



In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(30, 10))
for i, ax in enumerate(axes.flatten()):
    sns.heatmap(targets[i].squeeze(), cmap=cmap, ax=ax)
    ax.axis('off')
    ax.scatter(electrodes[i][:,0], electrodes[i][:,1], marker='x', c='r', legend='Electrodes')

# Load from datasets

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
dataset = torch.load('/home/nibdombe/deep_eit/data/datasets/train_dataset.pt')

In [ ]:
data = np.load('/home/nibdombe/deep_eit/data/processed/3d/case_TCIA_194_0/case_TCIA_194_0_5.npz')

In [ ]:
points = data['points'][0]
targets = data['targets'][0]
electrodes = data['electrodes'][0][:,0, 0]

### random translation

In [ ]:
cmap = make_cmap()
n_col = 4
fig, axes = plt.subplots(1, n_col, figsize=(int(n_col*5), 5))
translations_x = [0.05, -0.1, 0.1, -0.05]
translations_y = [-0.1, 0.05, 0.05, 0.1]
translations_z = [0, 0, 0, 0]
for i, ax in enumerate(axes.flatten()):
    scale = 0.025
    translation_x = translations_x[i]
    translation_y = translations_y[i]
    translation_z = translations_z[i]
    translation = np.array([translation_x, translation_y, translation_z]).reshape(3)
    trans_points = points + translation
    trans_electrode = electrodes.copy() + translation
    # ax.imshow(t.squeeze())
    ax.scatter(trans_points[:,0], -1*trans_points[:,1], marker='x', c=targets.flatten(), s=.1, cmap=cmap)
    ax.scatter(trans_electrode[:,0], -1*trans_electrode[:,1], marker='x', c='r', s=20, label='Electrodes')
    ax.set_xlim(-0.3, 0.3)
    ax.set_ylim(-0.3, 0.3)
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    ax.set_xlabel('')
    ax.set_ylabel('')
    if i==0:
        ax.legend()

In [ ]:
electrodes.shape

In [ ]:
cmap = make_cmap()
n_col = 4
fig, axes = plt.subplots(1, n_col, figsize=(int(n_col*5), 5))
rotation_angles = np.linspace(0, 2 * np.pi, n_col+2)[1:]


for i, ax in enumerate(axes.flatten()):
    rotation_angle = rotation_angles[i]
    rotation_matrix = np.array([[np.cos(rotation_angle), -np.sin(rotation_angle)],
                                    [np.sin(rotation_angle), np.cos(rotation_angle)]])
    
    rot_points = points.copy()
    rot_points[:,:2] = np.matmul(rot_points[:,:2], rotation_matrix)
    rot_electrode = electrodes.copy()
    rot_electrode[:,:2] = np.matmul(rot_electrode[:,:2], rotation_matrix)
    # ax.imshow(t.squeeze())
    ax.scatter(rot_points[:,0], -1*rot_points[:,1], marker='x', c=targets.flatten(), s=.1, cmap=cmap)
    ax.scatter(rot_electrode[:,0], -1*rot_electrode[:,1], marker='x', c='r', s=20, label='Electrodes')
    # ax.scatter(c[:,0], -1*c[:,1], marker='o', c='y', s=20, label='E-Center')
    # ax.scatter(0, 0, marker='o', c='white', edgecolors='black', s=60, label='Rotation Origin')
    # ax.annotate('', xy=(e[0,0], -1*e[0,1]), xytext=(0, 0),
    #              arrowprops=dict(facecolor='red', shrink=0.1))
    # ax.annotate('', xy=(e[1,0], -1*e[1,1]), xytext=(0, 0),
    #              arrowprops=dict(facecolor='red', shrink=0.1))
    # ax.annotate('', xy=(c[0,0], -1*c[0,1]), xytext=(0, 0),
    #              arrowprops=dict(facecolor='y', shrink=0.1))
    ax.set_xlim(-0.3, 0.3)
    ax.set_ylim(-0.3, 0.3)
    ax.xaxis.set_ticks([])
    ax.yaxis.set_ticks([])
    ax.set_xlabel('')
    ax.set_ylabel('')
    if i==0:
        ax.legend()

In [ ]:
def show_rotation(angles):
    if isinstance(angles, int):
        angles = [angles]
    fig, axes = plt.subplots(1, len(angles), figsize=(int(5*len(angles)), 5))
    if len(angles) == 1:
        axes = [axes]
    for i, angle in enumerate(angles):
        rad_angle = torch.deg2rad(torch.tensor(angle))
        rot = dataset._random_rotation_matrix(rotation_angle=rad_angle)
        e_rot = torch.matmul(e, rot)
        p_rot = torch.matmul(p[:,:2].float(), rot)
        c_rot = torch.matmul(torch.from_numpy(c), rot)

        # fig, ax = plt.subplots(figsize=(5, 5))
        # ax.imshow(t.squeeze())
        axes[i].scatter(p_rot[:,0], -1*p_rot[:,1], marker='x', c=t.flatten(), s=.1, cmap=cmap)
        axes[i].scatter(e_rot[:,0], -1*e_rot[:,1], marker='x', c='r', s=20, label='Electrodes')
        axes[i].scatter(c_rot[:,0], -1*c_rot[:,1], marker='o', c='y', s=20, label='E-Center')
        axes[i].scatter(0, 0, marker='o', c='white', edgecolors='black', s=60, label='Rotation Origin')
        axes[i].annotate('', xy=(e_rot[0,0], -1*e_rot[0,1]), xytext=(0, 0),
                    arrowprops=dict(facecolor='red', shrink=0.1))
        axes[i].annotate('', xy=(e_rot[1,0], -1*e_rot[1,1]), xytext=(0, 0),
                    arrowprops=dict(facecolor='red', shrink=0.1))
        axes[i].annotate('', xy=(c_rot[0,0], -1*c_rot[0,1]), xytext=(0, 0),
                    arrowprops=dict(facecolor='y', shrink=0.1))
        axes[i].set_xlim(-1, 1)
        axes[i].set_ylim(-1, 1)
        axes[i]. set_title(f'Rotation Angle: {angle}')
        if i == len(angles)-1:
            axes[i].legend()


In [ ]:
show_rotation([0, 90, 180,360])

In [ ]:
from torch.utils.data import DataLoader
dl = DataLoader(dataset, batch_size=3, shuffle=True)

In [ ]:
for p, _, s, e, m, t in dl:
    points = p
    targets = t
    electrodes = e
    break

In [ ]:
rot = dataset._random_rotation_matrix(torch.deg2rad(torch.tensor(90)))

In [ ]:
plt.scatter(electrodes[0,:,0,0,0], electrodes[0,:,0,0,1], marker='x', c='r')
plt.scatter(points[0,::2,0], points[0,::2,1], marker='x',s=0.01, c=targets[0,::2])

plt.axis('equal')
plt.xlim(-1,1)
plt.ylim(-1,1)


In [ ]:
electrodes_rot = torch.matmul(electrodes[:,:,:,:,:2], rot)
points_rot = torch.matmul(points, rot)

In [ ]:
plt.scatter(electrodes_rot[0,:,0,0,0], electrodes_rot[0,:,0,0,1], marker='x', c='r')
plt.scatter(points_rot[0,::2,0], points_rot[0,::2,1], marker='x',s=0.01, c=targets[0,::2])
plt.axis('equal')
plt.xlim(-1,1)
plt.ylim(-1,1)


In [ ]:
points.shape

# Compare generated targets with Anita's targets

In [ ]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
toy = np.load('/work/deep_eit/data/processed/case_TCIA_53_0.npz', allow_pickle=True)

In [ ]:
toy['targets'].shape

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
sns.heatmap(toy['targets'][0].squeeze(), ax=axes[0])
axes[0].axis('off')
sns.heatmap(toy['targets'][1].squeeze(), ax=axes[1])

In [ ]:
anita_target = sio.loadmat('/work/deep_eit/data/raw/case_TCIA_53_0/targets/level_1.mat')['slice_z1']
anita_target = np.where(np.isnan(anita_target), 0, anita_target)
anita_target = np.moveaxis(anita_target, 0, 1)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))
sns.heatmap(toy['targets'][0].squeeze(), ax=axes[0], vmax=1)
axes[0].axis('off')
sns.heatmap(anita_target, ax=axes[1], vmax=1)
axes[1].axis('off')


In [ ]:
anita_target.shape

In [ ]:
np.unique(anita_target)

In [ ]:
import os


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import torchvision.transforms.functional as TF

In [ ]:
from data_processing.mesh_to_array import mesh_to_image

In [ ]:
mesh = '/home/nibdombe/deep_eit/data/raw/case_TCIA_21_0/shape/mesh.vtk'   

In [ ]:
res = mesh_to_image(mesh, resolution=512)

In [ ]:
points = np.asarray(res[1].points)

In [ ]:
points.shape

In [ ]:
points_re = points.reshape(512,512,3)

In [ ]:
res[0].shape

In [ ]:
plt.scatter(points[:,0], points[:,1], c=res[0].flatten())

In [ ]:
points_res = points - np.mean(points, axis=0)
plt.scatter(points_res[:,0], points_res[:,1], c=res[0].flatten())

In [ ]:
from model.positional_encoding import positional_encoding
import torch

In [ ]:
coord = torch.tensor([[torch.pi,-torch.pi,torch.pi, 1],
                     [4,5,6,5],
                     [7,8,9,29]])
print(coord.shape)                     

In [ ]:
coord_pe = positional_encoding(coord, num_encoding_functions=2)
print(coord_pe.shape)

In [ ]:
coord_pe[0]

In [ ]:
coord_pe_0 = positional_encoding(coord[0], num_encoding_functions=2)

In [ ]:
print(coord_pe_0)
print(coord_pe[0])

In [ ]:
coord_more_dim = positional_encoding(coord.reshape(3,2,2), num_encoding_functions=2)

In [ ]:
torch.all(coord_pe_0 == coord_more_dim[0].flatten())

In [ ]:
coord_more_dim_pe = coord_more_dim[0].flatten()

In [ ]:
torch.sort(coord_more_dim_pe)[0] == torch.sort(coord_pe_0)[0]

In [2]:
test_3 = np.load('/home/nibdombe/deep_eit/data/processed/3d_mask/case_TCIA_401_0/case_TCIA_401_0_3.npz')
test_5 = np.load('/home/nibdombe/deep_eit/data/processed/3d_mask/case_TCIA_401_0/case_TCIA_401_0_5.npz')

In [4]:
signals_3 = test_3['signals']
signals_5 = test_5['signals']

In [6]:
signals_5

array([[5.84431362e+00, 2.02179313e+00, 3.34397435e-01, 3.68007928e-01,
        7.94319153e-01, 1.36940098e+00, 7.67001629e-01, 3.00924599e-01,
        1.20110720e-01, 1.34531289e-01, 4.01301116e-01, 1.21459997e+00,
        3.36822629e+00, 6.45116091e+00, 6.26429439e-01, 5.72005272e-01,
        1.01035130e+00, 1.10180974e+00, 3.60253483e-01, 1.12524137e-01,
        3.60650495e-02, 3.51574868e-02, 9.67350155e-02, 1.76984757e-01,
        2.84602940e-01, 4.07610083e+00, 2.55628538e+00, 1.34808922e+00,
        1.78457558e+00, 1.33270335e+00, 3.50975662e-01, 1.06635928e-01,
        3.07441913e-02, 2.63883546e-02, 6.78959861e-02, 1.03753939e-01,
        1.38307706e-01, 1.06090832e+00, 5.84417820e+00, 6.63628340e+00,
        5.04123068e+00, 1.98420215e+00, 4.01203662e-01, 1.20468840e-01,
        3.37584876e-02, 2.78918557e-02, 7.14212209e-02, 9.66777876e-02,
        1.16913952e-01, 6.03468657e-01, 2.02188468e+00, 6.45093155e+00,
        8.91616917e+00, 1.22566748e+00, 1.73748195e-01, 4.351332